In [5]:
%%capture
!pip install pandas
!pip install numpy
!pip install spacy
!pip install networkx
!pip install matplotlib.pyplot
!pip install re
!pip install os

import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import networkx as nx
import matplotlib.pyplot as plt
import os
import re

!python -m spacy download en_core_web_sm
# installing and importing libraries needed

In [6]:
# function definitions

def load_books(book_folder):
    texts = []
    for file_name in os.listdir(book_folder):
        file_path = os.path.join(book_folder, file_name)
    
        # Check if the path is a file (not a directory)
        if os.path.isfile(file_path):
            texts.append(file_path)
    return texts

def filter_entity(ent_list, character_df):
    return [ent for ent in ent_list
            if ent in list(character_df.character)
            or ent in list(character_df.character_firstname)
            or any(ent in alias_list for alias_list in character_df.aliases)]

def extract_last_name(full_name, last_names):
    for last_name in last_names:
        if last_name in full_name[0]:
            return last_name
    return full_name[0]

In [7]:
# load in text and langauge model

NER = spacy.load("en_core_web_sm")
texts = load_books("book_split_output")
# for text in texts:
#     book_doc = NER(open(text, 'r', encoding='utf-8').read())
book_doc = NER(open(texts[0], 'r', encoding='utf-8').read())

In [8]:
# load in names of characters
character_df = pd.read_csv("names.csv")


# add columns for different variations of name
character_df["character"] = character_df["character"].apply(lambda x: re.sub(r"[^\w\s]+", "", x))
character_df["character_firstname"] = character_df["character"].apply(lambda x: x.split(" ", 1) [0])
character_df["character_lastname"] = character_df["character"].apply(lambda x: x.split(" ", 1) [-1])
character_df["aliases"] = character_df["aliases"].apply(lambda x: [] if isinstance(x, float) else x.split(","))

# use NER to process and grab the list of entities and their sentences
sent_entity_df = []

for sent in book_doc.sents:
    entity_list = [ent.text for ent in sent.ents]
    sent_entity_df.append({"sentence": sent, "entities": entity_list})

# turn list into df
sent_entity_df = pd.DataFrame(sent_entity_df)

# filter out only the character names
sent_entity_df["character_entities"] = sent_entity_df["entities"].apply(lambda x: filter_entity(x, character_df))

# filter out sentences that don't have any entities in them
sent_entity_df_filtered = sent_entity_df[sent_entity_df["character_entities"].map(len) > 0]

# set entity name to last name of entity to aggregate same names i.e. Kim Dokja vs Dokja should be counted as same
sent_entity_df_filtered['character_entities'] = sent_entity_df_filtered.apply(lambda row: [extract_last_name(row['character_entities'], character_df['character_lastname'])], axis=1)
pd.reset_option('^display.', silent = True)

C:\Users\tommy\AppData\Local\Temp\ipykernel_11992\2167441617.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sent_entity_df_filtered['character_entities'] = sent_entity_df_filtered.apply(lambda row: [extract_last_name(row['character_entities'], character_df['character_lastname'])], axis=1)


In [9]:
# scroll through 5 lines at a time to aggregate relationships between characters and reorder to make sure A -> B is the same as B -> A
window_size = 5
relationships = []

for i in range(sent_entity_df_filtered.index[-1]):
    end_i = min(i+5, sent_entity_df_filtered.index[-1])
    char_list = sum((sent_entity_df_filtered.loc[i : end_i].character_entities), [])

    char_unique = [char_list[i] for i in range(len(char_list))
                   if (i == 0) or char_list[i] != char_list[i-1]]
    if len(char_unique) > 1:
        for idx, a in enumerate(char_unique[:-1]):
            b = char_unique[idx + 1]
            relationships.append({"source": a, "target": b})


In [10]:
# count the occurences of relationships between characters
relationship_df = pd.DataFrame(relationships)
relationship_df.head(10)
relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)

relationship_df["value"] = 1
relationship_df = relationship_df.groupby(["source", "target"], sort=False, as_index = False). sum()

In [11]:
relationship_df.head(10)

,source,target,value
0,Dokja,Sangah,4
1,Kang,Sangah,5
2,Hyunsung,Sangah,99
3,Hyunsung,Namwoon,17
4,Han,Namwoon,24
5,Namwoon,Sangah,3
6,Namwoon,One,5
7,Dokja,Hyunsung,6
8,Gilyoung,Sangah,80
9,Black Flame Dragon,Demon,1
